In [1]:
# Install necessary packages
!pip install --quiet -U langchain_openai langchain_core langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00


In [2]:
# Import necessary modules
import os
import getpass


In [3]:
def _set_env(var: str, value: str = None):
    if value:
        os.environ[var] = value
    elif not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


In [4]:
# Set API keys
_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")


OPENAI_API_KEY: ··········
LANGCHAIN_API_KEY: ··········


In [5]:
# Set LangChain tracing (optional)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "dream-interpretation-agent"


In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)


In [7]:
from pydantic import BaseModel, Field
from typing import List, Optional


In [8]:
class DreamSymbol(BaseModel):
    symbol: str = Field(description="Symbol extracted from the dream.")
    context: str = Field(description="Context or description of the symbol within the dream.")


In [9]:
class DreamAnalyst(BaseModel):
    name: str = Field(description="Name of the dream analyst.")
    approach: str = Field(description="Theoretical approach (e.g., Freudian, Jungian, Cognitive).")
    experience_years: int = Field(description="Number of years of experience in dream analysis.")
    personality_traits: List[str] = Field(description="Personality traits that affect interpretation style.")
    description: str = Field(description="Detailed description of the analyst's focus and interpretation style.")


In [10]:
class AnalystTeam(BaseModel):
    analysts: List[DreamAnalyst] = Field(
        description="List of dream analysts with their approaches and expertise."
    )


In [11]:
class SymbolList(BaseModel):
    symbols: List[DreamSymbol] = Field(description="List of extracted dream symbols.")


In [12]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


In [13]:
def merge_symbols(left: List[DreamSymbol] | None, right: List[DreamSymbol] | None) -> List[DreamSymbol]:
    if left is None:
        left = []
    if right is None:
        right = []
    # Merge without duplicates based on the symbol
    symbols = {symbol.symbol: symbol for symbol in left + right}
    return list(symbols.values())


In [14]:
def merge_interpretation_sections(left: List[str] | None, right: List[str] | None) -> List[str]:
    if left is None:
        left = []
    if right is None:
        right = []
    return left + right


In [15]:
class DreamState(TypedDict):
    user_id: str
    messages: Annotated[List[AnyMessage], add_messages]
    dream_description: str
    symbols: Annotated[List[DreamSymbol], merge_symbols]
    analysts: List[DreamAnalyst]
    interpretation_sections: Annotated[List[str], merge_interpretation_sections]
    final_report: str
    max_analysts: int


##System Messages and Instructions

In [18]:
#a. Symbol Extraction Instructions

In [16]:
symbol_extraction_instructions = """
You are a dream analysis assistant. Your task is to extract significant symbols from the dream description provided.

Instructions:
1. Identify all notable symbols, objects, characters, or events mentioned in the dream.
2. For each symbol, provide the context or description within the dream.
3. Do not provide interpretations at this stage.
4. Present the symbols in a structured format as per the SymbolList model.
"""


b. Analyst Generation Instructions

In [17]:
analyst_generation_instructions = """
You are tasked with creating a team of dream analyst personas for interpreting the dream. Follow these instructions carefully:

1. Review the dream description:
{dream_description}

2. Determine the key themes or elements that require interpretation.

3. Create up to {max_analysts} analysts, each specializing in a different theoretical approach relevant to dream analysis.

4. For each analyst, provide the following:
   - Name
   - Theoretical approach (e.g., Freudian, Jungian, Cognitive)
   - Number of years of experience in dream analysis
   - Personality traits that affect their interpretation style (e.g., analytical, empathetic, creative)
   - A detailed description of their focus and interpretation style

5. Ensure that the team is diverse in expertise and perspective to provide comprehensive interpretations.
"""


c. Symbol Analysis Instructions Template

In [19]:
symbol_analysis_instructions_template = """
You are {analyst_name}, a dream analyst specializing in {approach} with {experience_years} years of experience. Your personality traits are: {personality_traits}.

Instructions:
1. Analyze the following dream symbol: "{symbol}" in the context: "{context}".
2. Provide a detailed interpretation based on your theoretical approach.
3. Use your personality traits to influence your interpretation style.
4. Do not reference other analysts or interpretations.
"""


In [20]:
from langchain_core.messages import SystemMessage, HumanMessage

def extract_symbols(state: DreamState):
    dream_description = state['dream_description']

    # Enforce structured output
    structured_llm = llm.with_structured_output(SymbolList)

    # System message
    system_message = SystemMessage(content=symbol_extraction_instructions)

    # Human message
    human_message = HumanMessage(content=dream_description)

    # Invoke LLM
    response = structured_llm.invoke([system_message, human_message])

    # Extract symbols from response
    extracted_symbols = response.symbols

    return {'symbols': extracted_symbols}


In [21]:
def create_analysts(state: DreamState):
    dream_description = state['dream_description']
    max_analysts = state.get('max_analysts', 3)

    # Enforce structured output
    structured_llm = llm.with_structured_output(AnalystTeam)

    # System message
    system_message_content = analyst_generation_instructions.format(
        dream_description=dream_description,
        max_analysts=max_analysts
    )

    # Generate analysts
    analyst_team = structured_llm.invoke([
        SystemMessage(content=system_message_content),
        HumanMessage(content="Generate the analyst personas.")
    ])

    # Write the list of analysts to state
    return {"analysts": analyst_team.analysts}


In [22]:
def analyze_symbols(state: DreamState):
    symbols = state['symbols']
    analysts = state['analysts']
    interpretation_sections = []

    num_analysts = len(analysts)
    for i, symbol in enumerate(symbols):
        analyst = analysts[i % num_analysts]

        # Prepare system message with analyst persona
        personality_traits = ', '.join(analyst.personality_traits)
        system_message_content = symbol_analysis_instructions_template.format(
            analyst_name=analyst.name,
            approach=analyst.approach,
            experience_years=analyst.experience_years,
            personality_traits=personality_traits,
            symbol=symbol.symbol,
            context=symbol.context
        )

        system_message = SystemMessage(content=system_message_content)

        # Generate interpretation
        interpretation = llm.invoke([system_message])

        # Store the interpretation section
        interpretation_section = f"**Analyst:** {analyst.name}\n**Approach:** {analyst.approach}\n**Symbol:** {symbol.symbol}\n{interpretation.content}\n"
        interpretation_sections.append(interpretation_section)

    # Update state
    return {
        'interpretation_sections': interpretation_sections
    }


In [23]:
def compile_dream_report(state: DreamState):
    interpretation_sections = state.get('interpretation_sections', [])

    report = "# Dream Interpretation Report\n\n## Interpretations\n\n"
    report += "\n".join(interpretation_sections)

    return {'final_report': report}


In [24]:
# Build the main graph
from langgraph.graph import StateGraph, START, END

builder = StateGraph(DreamState)
builder.add_node('extract_symbols', extract_symbols)
builder.add_node('create_analysts', create_analysts)
builder.add_node('analyze_symbols', analyze_symbols)
builder.add_node('compile_dream_report', compile_dream_report)

# Edges
builder.add_edge(START, 'extract_symbols')
builder.add_edge('extract_symbols', 'create_analysts')
builder.add_edge('create_analysts', 'analyze_symbols')
builder.add_edge('analyze_symbols', 'compile_dream_report')
builder.add_edge('compile_dream_report', END)


In [25]:
# Compile the graph
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

dream_graph = builder.compile(checkpointer=memory)


In [26]:
# Initial state
initial_state = {
    'user_id': 'dreamer1',
    'dream_description': (
        'I was walking through a dense forest when I stumbled upon a hidden door in a tree. '
        'I opened the door and found a room filled with clocks all ticking at different times. '
        'Suddenly, a white rabbit appeared and gestured me to follow it.'
    ),
    'messages': [],
    'max_analysts': 3
}


In [27]:
# Config
config = {'configurable': {'thread_id': 'dream_thread1'}}


In [28]:
# Execute the graph
result_state = dream_graph.invoke(initial_state, config)


In [29]:
# Get the final state
final_state = dream_graph.get_state(config)[0]


In [30]:
# Pretty print the final report
from IPython.display import display, Markdown

print("\nDream Interpretation Report:")
display(Markdown(final_state['final_report']))



Dream Interpretation Report:


# Dream Interpretation Report

## Interpretations

**Analyst:** Dr. Amelia Freudson
**Approach:** Freudian
**Symbol:** dense forest
In Freudian dream analysis, every element of a dream is believed to represent an aspect of the dreamer's unconscious mind. The "dense forest" in this context can be seen as a symbol of the dreamer's unconscious mind itself. Forests are often associated with mystery, unknown, and complexity, much like our unconscious mind. The density of the forest suggests that the dreamer's unconscious mind is filled with numerous unresolved issues, repressed emotions, or hidden desires that are yet to be explored or understood.

Walking through this dense forest indicates the dreamer's journey into their own unconscious mind. This could be a sign that the dreamer is currently in a phase of self-discovery or introspection, trying to navigate through their complex emotions, past experiences, or hidden desires. The dreamer might be trying to understand their true self, their fears, or their desires that they have repressed or ignored in their waking life.

However, the interpretation of this dream symbol can also depend on the dreamer's feelings and experiences in the dream. If the dreamer felt lost or scared in the forest, it might indicate their fear or anxiety towards exploring their unconscious mind or confronting their repressed emotions or desires. On the other hand, if the dreamer felt calm or curious, it might suggest their readiness or eagerness to understand their unconscious mind and resolve their inner conflicts.

As an analyst, I meticulously examine every detail of the dream and its context to provide a comprehensive interpretation. I believe that understanding our dreams can help us understand our unconscious mind and thus, ourselves better. However, it's important to remember that this interpretation is based on Freudian theory and the dreamer's personal experiences and feelings can significantly influence the meaning of their dream.

**Analyst:** Prof. Carl Jungman
**Approach:** Jungian
**Symbol:** hidden door in a tree
In Jungian dream analysis, every symbol in a dream is a reflection of the dreamer's subconscious mind. The "hidden door in a tree" is a particularly rich symbol, suggesting a deep exploration of the self and the natural world.

The tree, in many cultures and psychological theories, is a symbol of life, growth, and the interconnectedness of all things. It represents the dreamer's personal growth and development. The fact that the tree houses a hidden door suggests that there is an aspect of the dreamer's personal growth or life experience that they have not yet fully explored or understood.

The door is a common symbol in dreams, often representing a transition or opportunity. A hidden door, however, suggests a secret or unexplored aspect of the dreamer's life or personality. It could represent a hidden potential, a secret desire, or an aspect of the dreamer's personality that they have not yet acknowledged or accepted.

The act of stumbling upon this hidden door suggests that the dreamer is not actively seeking this hidden aspect of themselves, but has come across it unexpectedly. This could indicate that the dreamer is in a phase of their life where they are beginning to uncover and explore aspects of their personality or life experience that they were previously unaware of.

In the context of the dream, the hidden door in the tree could be a call from the subconscious to explore these hidden aspects of the self. It suggests a need for introspection and self-discovery. The dreamer may be ready to delve deeper into their own psyche, to uncover and understand their true nature and potential.

As an intuitive, empathetic, and creative dream analyst, I would encourage the dreamer to embrace this journey of self-discovery. I would suggest that they pay attention to their feelings and intuition as they explore this hidden aspect of themselves. They may find that this journey leads to a deeper understanding of themselves and their place in the world.

**Analyst:** Dr. Lisa Cognita
**Approach:** Cognitive
**Symbol:** room filled with clocks
Interpreting the symbol of a "room filled with clocks" in the context of the dream, it's important to consider the cognitive perspective, which emphasizes the role of thought processes and the individual's understanding of themselves and their world.

A room filled with clocks, all ticking at different times, can be seen as a representation of the dreamer's perception of time and their life events. Clocks are universally recognized symbols of time, order, and the rhythm of life. However, the fact that these clocks are all ticking at different times suggests a sense of disarray or confusion. This could indicate that the dreamer is experiencing a feeling of being out of sync with their own life timeline or with the timelines of others around them.

The room itself can be seen as a symbol of the dreamer's mind or personal space. The presence of so many clocks could suggest an overwhelming preoccupation with time, deadlines, or aging. It could also indicate a feeling of being surrounded by or trapped by these concerns.

From a cognitive perspective, this dream could be a reflection of the dreamer's current mental state. If they are feeling stressed or anxious about certain deadlines or life events, their mind might be manifesting these concerns in the form of this dream. The different times on the clocks could represent different tasks or responsibilities that the dreamer is juggling, and the feeling of being out of sync could reflect their struggle to manage these tasks effectively.

In conclusion, this dream seems to be a manifestation of the dreamer's concerns about time and their ability to manage their responsibilities. It might be beneficial for the dreamer to examine their current stressors and consider whether they need to take steps to better manage their time or reduce their workload.

**Analyst:** Dr. Amelia Freudson
**Approach:** Freudian
**Symbol:** white rabbit
In Freudian dream analysis, every symbol in a dream is a manifestation of the dreamer's subconscious mind. The "white rabbit" in this context is a particularly intriguing symbol. 

White, as a color, often represents purity, innocence, and in some cases, enlightenment. Rabbits, on the other hand, are typically associated with fertility, abundance, and fear due to their timid nature. However, the rabbit's gesture to follow it suggests a leadership or guiding role, which is not typically associated with the creature. 

Analyzing this meticulously, the white rabbit could symbolize a part of the dreamer's subconscious that is pure and innocent, yet filled with potential (fertility) and perhaps a bit of fear or apprehension (rabbit's timid nature). The act of the rabbit gesturing the dreamer to follow it could represent an invitation from the dreamer's subconscious to delve deeper into their own innocence, potential, and fears.

The dreamer might be at a point in their life where they are being called to explore these aspects of themselves. This could be a journey of self-discovery, personal growth, or even a confrontation with fears that have been long suppressed. 

In a more introspective view, the dreamer might be yearning for a return to innocence or purity, which the white rabbit symbolizes. This could be a subconscious desire to retreat from the complexities and disappointments of adult life, seeking solace in the simplicity and abundance of childhood or a more innocent time.

In conclusion, the white rabbit in this dream is a complex symbol that represents a call to introspection, a journey into the self, and a confrontation with one's fears and potential. It's an invitation to explore the purity and abundance within, despite the fear that such a journey might entail.


In [31]:
pip install --quiet gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.7 MB/s eta 0:00:00


In [32]:
import gradio as gr


In [33]:
# Install necessary packages
!pip install --quiet -U langchain_openai langchain_core langgraph gradio

# Import necessary modules
import os
import getpass
import gradio as gr

def _set_env(var: str, value: str = None):
    if value:
        os.environ[var] = value
    elif not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Enter your {var}: ")

# Set API keys
_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

# Set LangChain tracing (optional)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "dream-interpretation-agent"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define data models
from pydantic import BaseModel, Field
from typing import List, Optional

class DreamSymbol(BaseModel):
    symbol: str = Field(description="Symbol extracted from the dream.")
    context: str = Field(description="Context or description of the symbol within the dream.")

class DreamAnalyst(BaseModel):
    name: str = Field(description="Name of the dream analyst.")
    approach: str = Field(description="Theoretical approach (e.g., Freudian, Jungian, Cognitive).")
    experience_years: int = Field(description="Number of years of experience in dream analysis.")
    personality_traits: List[str] = Field(description="Personality traits that affect interpretation style.")
    description: str = Field(description="Detailed description of the analyst's focus and interpretation style.")

class AnalystTeam(BaseModel):
    analysts: List[DreamAnalyst] = Field(
        description="List of dream analysts with their approaches and expertise."
    )

class SymbolList(BaseModel):
    symbols: List[DreamSymbol] = Field(description="List of extracted dream symbols.")

# Define state schema and helper functions
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

def merge_symbols(left: List[DreamSymbol] | None, right: List[DreamSymbol] | None) -> List[DreamSymbol]:
    if left is None:
        left = []
    if right is None:
        right = []
    # Merge without duplicates based on the symbol
    symbols = {symbol.symbol: symbol for symbol in left + right}
    return list(symbols.values())

def merge_interpretation_sections(left: List[str] | None, right: List[str] | None) -> List[str]:
    if left is None:
        left = []
    if right is None:
        right = []
    return left + right

class DreamState(TypedDict):
    user_id: str
    messages: Annotated[List[AnyMessage], add_messages]
    dream_description: str
    symbols: Annotated[List[DreamSymbol], merge_symbols]
    analysts: List[DreamAnalyst]
    interpretation_sections: Annotated[List[str], merge_interpretation_sections]
    final_report: str
    max_analysts: int

# System messages and instructions
symbol_extraction_instructions = """
You are a dream analysis assistant. Your task is to extract significant symbols from the dream description provided.

Instructions:
1. Identify all notable symbols, objects, characters, or events mentioned in the dream.
2. For each symbol, provide the context or description within the dream.
3. Do not provide interpretations at this stage.
4. Present the symbols in a structured format as per the SymbolList model.
"""

analyst_generation_instructions = """
You are tasked with creating a team of dream analyst personas for interpreting the dream. Follow these instructions carefully:

1. Review the dream description:
{dream_description}

2. Determine the key themes or elements that require interpretation.

3. Create up to {max_analysts} analysts, each specializing in a different theoretical approach relevant to dream analysis.

4. For each analyst, provide the following:
   - Name
   - Theoretical approach (e.g., Freudian, Jungian, Cognitive)
   - Number of years of experience in dream analysis
   - Personality traits that affect their interpretation style (e.g., analytical, empathetic, creative)
   - A detailed description of their focus and interpretation style

5. Ensure that the team is diverse in expertise and perspective to provide comprehensive interpretations.
"""

symbol_analysis_instructions_template = """
You are {analyst_name}, a dream analyst specializing in {approach} with {experience_years} years of experience. Your personality traits are: {personality_traits}.

Instructions:
1. Analyze the following dream symbol: "{symbol}" in the context: "{context}".
2. Provide a detailed interpretation based on your theoretical approach.
3. Use your personality traits to influence your interpretation style.
4. Do not reference other analysts or interpretations.
"""

# Define functions and nodes
from langchain_core.messages import SystemMessage, HumanMessage

def extract_symbols(state: DreamState):
    dream_description = state['dream_description']

    # Enforce structured output
    structured_llm = llm.with_structured_output(SymbolList)

    # System message
    system_message = SystemMessage(content=symbol_extraction_instructions)

    # Human message
    human_message = HumanMessage(content=dream_description)

    # Invoke LLM
    response = structured_llm.invoke([system_message, human_message])

    # Extract symbols from response
    extracted_symbols = response.symbols

    return {'symbols': extracted_symbols}

def create_analysts(state: DreamState):
    dream_description = state['dream_description']
    max_analysts = state.get('max_analysts', 3)

    # Enforce structured output
    structured_llm = llm.with_structured_output(AnalystTeam)

    # System message
    system_message_content = analyst_generation_instructions.format(
        dream_description=dream_description,
        max_analysts=max_analysts
    )

    # Generate analysts
    analyst_team = structured_llm.invoke([
        SystemMessage(content=system_message_content),
        HumanMessage(content="Generate the analyst personas.")
    ])

    # Write the list of analysts to state
    return {"analysts": analyst_team.analysts}

def analyze_symbols(state: DreamState):
    symbols = state['symbols']
    analysts = state['analysts']
    interpretation_sections = []

    num_analysts = len(analysts)
    for i, symbol in enumerate(symbols):
        analyst = analysts[i % num_analysts]

        # Prepare system message with analyst persona
        personality_traits = ', '.join(analyst.personality_traits)
        system_message_content = symbol_analysis_instructions_template.format(
            analyst_name=analyst.name,
            approach=analyst.approach,
            experience_years=analyst.experience_years,
            personality_traits=personality_traits,
            symbol=symbol.symbol,
            context=symbol.context
        )

        system_message = SystemMessage(content=system_message_content)

        # Generate interpretation
        interpretation = llm.invoke([system_message])

        # Store the interpretation section
        interpretation_section = f"**Analyst:** {analyst.name}\n**Approach:** {analyst.approach}\n**Symbol:** {symbol.symbol}\n{interpretation.content}\n"
        interpretation_sections.append(interpretation_section)

    # Update state
    return {
        'interpretation_sections': interpretation_sections
    }

def compile_dream_report(state: DreamState):
    interpretation_sections = state.get('interpretation_sections', [])

    report = "# Dream Interpretation Report\n\n## Interpretations\n\n"
    report += "\n".join(interpretation_sections)

    return {'final_report': report}

# Build the main graph
from langgraph.graph import StateGraph, START, END

builder = StateGraph(DreamState)
builder.add_node('extract_symbols', extract_symbols)
builder.add_node('create_analysts', create_analysts)
builder.add_node('analyze_symbols', analyze_symbols)
builder.add_node('compile_dream_report', compile_dream_report)

# Edges
builder.add_edge(START, 'extract_symbols')
builder.add_edge('extract_symbols', 'create_analysts')
builder.add_edge('create_analysts', 'analyze_symbols')
builder.add_edge('analyze_symbols', 'compile_dream_report')
builder.add_edge('compile_dream_report', END)

# Compile the graph
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

dream_graph = builder.compile(checkpointer=memory)

# Define the Gradio app function
def run_dream_interpreter(dream_description, max_analysts):
    # Initial state
    initial_state = {
        'user_id': 'dreamer1',
        'dream_description': dream_description,
        'messages': [],
        'max_analysts': int(max_analysts)
    }

    # Config
    config = {'configurable': {'thread_id': 'dream_thread1'}}

    # Execute the graph
    try:
        result_state = dream_graph.invoke(initial_state, config)
    except Exception as e:
        return f"An error occurred during interpretation: {e}"

    # Get the final state
    final_state = dream_graph.get_state(config)[0]

    # Return the final report
    return final_state['final_report']

# Create the Gradio interface
iface = gr.Interface(
    fn=run_dream_interpreter,
    inputs=[
        gr.Textbox(
            lines=10,
            placeholder="Describe your dream here...",
            label="Dream Description"
        ),
        gr.Slider(
            minimum=1,
            maximum=5,
            step=1,
            value=3,
            label="Number of Analysts"
        )
    ],
    outputs="markdown",
    title="Dream Interpretation Agent",
    description="Enter your dream description to receive interpretations from AI-generated dream analysts.",
    allow_flagging="never"
)

# Launch the Gradio app
iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3917302fc1829e553.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
